In [1]:
import numpy as np
import os
import zarr

# Create a zarr hierarchy

In [75]:
store = zarr.DirectoryStore("example.zarr")
root = zarr.group(store=store, overwrite=True)
foo = root.create_group("foo")
foo.attrs["foo_attr1"] = True
foo.attrs["foo_attr2"] = np.nan
foo.attrs["foo_attr3"] = "NaN"
foo_data = foo.array("foo_data", data=[1, None], dtype="float", compressor=None, fill_value=np.nan)
bar = foo.create_group("bar")
bar_data = bar.zeros("bar_data", shape=(10, 10), chunks=(10, 10), dtype="i4")
bar_data.attrs["bar_data_attr1"] = 42
bar_data.attrs["bar_data_attr2"] = ["test1", "test2"]

In [76]:
f = open("example.zarr/foo/foo_data/.zarray")
print(f.read())

{
    "chunks": [
        2
    ],
    "compressor": null,
    "dtype": "<f8",
    "fill_value": "NaN",
    "filters": null,
    "order": "C",
    "shape": [
        2
    ],
    "zarr_format": 2
}


In [44]:
root.tree(expand=True)

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, name='foo', nodes=(Node(disabled=True, na…

In [45]:
store = zarr.consolidate_metadata(store)

In [46]:
os.listdir("example.zarr")

['.zgroup', 'foo', '.zmetadata']

In [47]:
# https://stackoverflow.com/a/59109706/20177
from pathlib import Path

# prefix components:
space =  '    '
branch = '│   '
# pointers:
tee =    '├── '
last =   '└── '


def tree(dir_path: Path, prefix: str=''):
    """A recursive generator, given a directory Path object
    will yield a visual tree structure line by line
    with each line prefixed by the same characters
    """    
    contents = list(dir_path.iterdir())
    # contents each get pointers that are ├── with a final └── :
    pointers = [tee] * (len(contents) - 1) + [last]
    for pointer, path in zip(pointers, contents):
        yield prefix + pointer + path.name
        if path.is_dir(): # extend the prefix and recurse:
            extension = branch if pointer == tee else space 
            # i.e. space because last, └── , above so no more |
            yield from tree(path, prefix=prefix+extension)

In [48]:
for line in tree(Path("example.zarr")):
    print(line)

├── .zgroup
├── foo
│   ├── .zattrs
│   ├── .zgroup
│   ├── foo_data
│   │   ├── .zarray
│   │   └── 0
│   └── bar
│       ├── .zgroup
│       └── bar_data
│           ├── .zarray
│           └── .zattrs
└── .zmetadata


# What does the zarr consolidated metadata look like?

In [49]:
import json
f = open("example.zarr/.zmetadata")
zmeta = json.load(f)
print(json.dumps(zmeta, indent=4))

{
    "metadata": {
        ".zgroup": {
            "zarr_format": 2
        },
        "foo/.zattrs": {
            "foo_attr1": true,
            "foo_attr2": NaN,
            "foo_attr3": "NaN"
        },
        "foo/.zgroup": {
            "zarr_format": 2
        },
        "foo/bar/.zgroup": {
            "zarr_format": 2
        },
        "foo/bar/bar_data/.zarray": {
            "chunks": [
                10,
                10
            ],
            "compressor": {
                "blocksize": 0,
                "clevel": 5,
                "cname": "lz4",
                "id": "blosc",
                "shuffle": 1
            },
            "dtype": "<i4",
            "fill_value": 0,
            "filters": null,
            "order": "C",
            "shape": [
                10,
                10
            ],
            "zarr_format": 2
        },
        "foo/bar/bar_data/.zattrs": {
            "bar_data_attr1": 42,
            "bar_data_attr2": [
            

# Test storing NaN as a zarr attribute

In [50]:
store = zarr.DirectoryStore("example-nan.zarr")
root = zarr.group(store=store, overwrite=True)
foo = root.attrs["foo_attr1"] = np.nan
f = open("example-nan.zarr/.zattrs")
print(f.read())

{
    "foo_attr1": NaN
}


In [ ]:
store = zarr.DirectoryStore("example.zarr")
root = zarr.group(store=store, overwrite=True)
root.array(data=None, dtype="float", fill_value=np.nan)
zarr.save("example-fill-value-nan.zarr", root)
f = open("example-fill-value-nan.zarr/.zarray")
print(f.read())

In [61]:
root = zarr.array(data=[1, 2, 3], shape=(5, 1), dtype="float", fill_value=np.nan)
zarr.save("example-fill-value-nan.zarr", root)
f = open("example-fill-value-nan.zarr/.zarray")
print(f.read())

{
    "chunks": [
        3
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<f8",
    "fill_value": 0.0,
    "filters": null,
    "order": "C",
    "shape": [
        3
    ],
    "zarr_format": 2
}


In [24]:
read_root = zarr.open("example-nan.zarr")
a = read_root.attrs["foo_attr1"]
print(a)
print(type(a))

nan
<class 'float'>


In [36]:
d = {"test": np.nan}
s = json.dumps(d)
print(json.loads(s)["test"])

nan


In [37]:
type(json.loads(s)["test"])

float

In [74]:
float('-inf')

-inf